# **Machine Learning Hackathon**

## **Topic: Energy Usage Prediction System**
    
**Problem statement:**
    
Create a methodology that utilizes data to **predict the electricity consumption** of
household appliances in low-energy homes, and **provide suggestions for optimizing
energy usage by identifying controllable factors.**

This approach will involve collecting data on various factors such as 
    a) appliance usage patterns *(data available)*, 
    b) weather conditions *(data available)*, and 
    c) other environmental factors *(data available)* 
that affect energy consumption. The data will be analyzed to determine trends and patterns that can inform recommendations for efficient energy usage.

These **recommendations may include adjusting the timing of appliance usage, implementing energy-efficient appliances, and taking advantage of renewable energy sources.** *(domain knowledge required)*

Ultimately, the goal is to help homeowners reduce their electricity bills and minimize
their environmental impact by maximizing the efficient use of energy.

Attribute Information:
    
1. date : time year-month-day hour:minute:second
2. Appliances : energy use ( transformed data), Logarithmic value from Wh
3. lights, energy use of light fixtures in the house in Wh
4. T1, Temperature in kitchen area, in Celsius
5. RH_1, Humidity in kitchen area, in %
6. T2, Temperature in living room area, in Celsius
7. RH_2, Humidity in living room area, in %
8. T3, Temperature in laundry room area
9. RH_3, Humidity in laundry room area, in %
10. T4, Temperature in office room, in Celsius
11. RH_4, Humidity in office room, in %
12. T5, Temperature in bathroom, in Celsius
13. RH_5, Humidity in bathroom, in %
14. T6, Temperature outside the building (north side), in Celsius
15. RH_6, Humidity outside the building (north side), in %
16. T7, Temperature in ironing room , in Celsius
17. RH_7, Humidity in ironing room, in %
18. T8, Temperature in teenager room 2, in Celsius
19. RH_8, Humidity in teenager room 2, in %
20. T9, Temperature in parents room, in Celsius
21. RH_9, Humidity in parents room, in %
22. To, Temperature outside (from XYZ weather station), in Celsius
23. Pressure (from XYZ weather station), in mm Hg
24. RH_out, Humidity outside (from XYZ weather station), in %
25. Wind speed (from XYZ weather station), in m/s
26. Visibility (from XYZ weather station), in km
27. Tdewpoint (from XYZ weather station), Â°C
28. rv1, Random variable 1, non dimensional
29. rv2, Random variable 2, non dimensional

2 goals:

1. Predicting Energy consumption patterns on unseen data
2. Identifying drivers of consumption and recommending the controllable features for optimizing consumption

# Workflow

- [1. Data Loading, Imports](#1)
- [2. Preprocessing](#2)
- [3. Feature Engineering,  Exploratory Data Analysis](#3)
    >[3.1 Univariate Analysis](#3.1)
    
    >[3.2 Bivariate Analysis](#3.2)
    
    >[3.3 Multivariate Analysis](#3.3)
    
    >[3.4 Summary Observations and Recommendations](#3.4)
- [4. Model Training, Hyperparameter tuning, Evaluation](#4)
- [5. Recommendations, EDA Summary, Model Summary](#5)


<a id="1"></a>
# 1. Data Loading, Imports


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from ydata_profiling import ProfileReport

# Set plot style
sns.set(color_codes=True)

plt.rcParams['figure.figsize']=15,8
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/household-energy-consumption-dataset/train.csv')
df_t = pd.read_csv('/kaggle/input/household-energy-consumption-dataset/test.csv')

In [ ]:
plt.figure(figsize=(25,12))
sns.heatmap(df.corr(method = 'spearman'), fmt = '.2f',  annot = True, mask = np.triu(df.corr()))
plt.savefig(f'corr-before-preprocessing.png')

<a id="2"></a>
# 2. Preprocessing

**Dataframe description**

In [ ]:
# lets start with the df_customer_demographic 
print('Shape:', df.shape)

print('='*80)
print('Information about dataset \n',df.info(),'\n')

print('='*80)
print('Null Values')
missing_indices = set()
for index, row in df.iterrows():
    if row.isnull().any():
        missing_indices.add(index)
(len(missing_indices), df.isna().sum().sum()),df.isna().sum()

In [ ]:
print('='*80)
print('First 5 rows \n')
df.head()

In [ ]:
print('='*80)
print('Describe the dataset \n')
df.describe()

**Duplicates**

In [ ]:
print('Duplicates count:\t',df.duplicated().sum())

**Outlier Analysis and Handling**

In [ ]:
## Checking for skewness/outliers
print('Feature Skewness \n','='*80)
df.skew()

In [ ]:
from scipy import stats
## Let's identify outliers, beyond 2 standard deviations from feature means
outlier_row_indices = set()
outlier_features = dict()
for column in df.drop(['ID','date'], axis =1).columns:
    z_scores = stats.zscore(df[column])
    outlier_row_indices.update(np.where(np.abs(z_scores) > 3)[0])
    outlier_features[column]=len(np.where(np.abs(z_scores) > 3)[0])
    
print('No ofrows with outlier attributes:\t',len(outlier_row_indices))
print('Percentage of rows with outlier attributes in dataset:',len(outlier_row_indices)/ df.shape[0])
print('\n', '='*80)
print('Feature wise outlier counts')
outlier_features

Based on psychrometric charts, features relative humidity is defined for a particular tempratre and hence t1-t9 and rh_1 to rh_9 are pair wise not correlated.


In [ ]:
numerical_columns = df.select_dtypes(include=['int64', 'float64']).drop('ID', axis = 1).drop('lights', axis = 1).drop('appliances', axis = 1).columns.tolist()


fig, axes = plt.subplots(nrows=20, ncols=4, figsize=(12, 50))

axes = axes.flatten()

for i, variable in enumerate(numerical_columns):
    sns.boxplot(x=df[variable], ax=axes[i]) 
    axes[i].set_title(variable) 
#     plt.savefig(f'boxplot-{variable}.jpg')
    
if len(numerical_columns) < len(axes):
    for j in range(len(numerical_columns), len(axes)):
        fig.delaxes(axes[j])

fig.tight_layout()

plt.show()


<a id="3"></a>
# 3. Feature Engineering,  Exploratory Data Analysis




**Reformat timestamp feature and extract date, month, week, day, hour**
- df
- df_t

In [ ]:
## df
df['month'] = pd.to_datetime(df['date']).dt.month
df['date_2'] = pd.to_datetime(df['date']).dt.date
df['time'] = pd.to_datetime(df['date']).dt.time
df['week'] = pd.to_datetime(df['date']).dt.week
df['day_num'] = pd.to_datetime(df['date']).dt.dayofweek
df['time'] = pd.to_datetime(df['date']).dt.time
df['day_type'] = pd.to_datetime(df['date']).dt.dayofweek.apply(lambda x: '0' if x < 5 else '1')
df['week_num'] = pd.to_datetime(df['date']).dt.week

df["hour"] = pd.to_datetime(df["date"]).dt.hour
df["weekday"] = pd.to_datetime(df["date"]).dt.weekday

In [ ]:
## df_t
df_t['month'] = pd.to_datetime(df_t['date']).dt.month
df_t['date_2'] = pd.to_datetime(df_t['date']).dt.date
df_t['time'] = pd.to_datetime(df_t['date']).dt.time
df_t['week'] = pd.to_datetime(df_t['date']).dt.week
df_t['day_num'] = pd.to_datetime(df_t['date']).dt.dayofweek
df_t['time'] = pd.to_datetime(df_t['date']).dt.time
df_t['day_type'] = pd.to_datetime(df_t['date']).dt.dayofweek.apply(lambda x: '0' if x < 5 else '1')
df_t['week_num'] = pd.to_datetime(df_t['date']).dt.week

df_t["hour"] = pd.to_datetime(df_t["date"]).dt.hour
df_t["weekday"] = pd.to_datetime(df_t["date"]).dt.weekday

In [ ]:
col_temp = ["t1","t2","t3","t4","t5","t6","t7","t8","t9"]

col_hum = ["rh_1","rh_2","rh_3","rh_4","rh_5","rh_6","rh_7","rh_8","rh_9"]

col_weather = ["t_out", "tdewpoint","rh_out","press_mm_hg",
                "windspeed","visibility"] 
col_light = ["lights"]

col_randoms = ["rv1", "rv2"]

col_time = ["hour", "weekday"]

col_target = ["appliances"]

In [ ]:
df[['day_num']].value_counts()

<a id="3.1"></a>
## 3.1 Univariate Analysis

Lets understand feature distributions

In [ ]:
len(df.columns)

In [ ]:
df.columns

In [ ]:
numerical_columns = df.select_dtypes(include=['int64', 'float64']).drop(['ID','month','rv1','rv2','visibility', 'windspeed',
       'week','day_num', 'week_num', 'lights'], axis = 1).columns.tolist()

fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(15, 20))

axes = axes.flatten()

for i, variable in enumerate(numerical_columns):
    df[variable].hist(ax=axes[i])
    axes[i].set_title(variable) 
    
fig.tight_layout()  
plt.show()

In [ ]:
df[df['month']==1].groupby('date_2').agg({'appliances':'mean'}).appliances

**Inferences**
1. **`rv1` and `rv2` features show no visible variance.** We'll verify their correlation strength against the target and make a retain/drop decision.
2. **`lights` feature has largely zero valuess.** We'll verify their correlation strength against the target and make a retain/drop decision.
3. **Temperature and Relative Humidity features'** histograms as captured by sensors 0-9 have **similar distributions.** From **psychrometric properties of ambient air,** relative humidity of air is defined for a given temperature. So it follows that, unless air is conditioned with humidity adjustments, **`t0-t9` and `rh_0-rh_9` should be pairwise correlated**.
4. **Windspeed and relative humidity move in the opposite directions** and the same is observed in weather station **`windspeed`** and **`rh_out`** features.
5. The day-wise consumption reveals some insights during winter and summer months of Jan and May. The transition spring months seems to see randm consumption.

<a id="3.2"></a>
## 3.2 Bivariate Analysis


**Daily consumption trendline**

In [ ]:
plt.style.use('ggplot')
months = [1, 2, 3, 4, 5]
m_names = ['January', 'February', 'March', 'April', 'May']
subplots = [511, 512, 513, 514, 515]

fig, axs = plt.subplots(len(months), 1, figsize=(15, 12))

for i, (month, m_name) in enumerate(zip(months, m_names)):
    ax = axs[i]
    df_month = df[df['month'] == month]
    sorted_dates = sorted(list(df_month['date_2'].unique()))

    # Get the mean appliance values for the current month
    y = df_month.groupby('date_2').agg({'appliances': 'mean'}).appliances.to_list()

    ax.plot(sorted_dates, y, color='green', linewidth=1.7)
    ax.set_title(m_name)

    # Set the x-axis tick positions and labels
    day_of_week_labels = [date.strftime('%a') for date in sorted_dates]
    day_of_month_labels = [date.day for date in sorted_dates] 
    
    x_labels = \
    [f'{dow}\n{dom}' for dow, dom in zip(day_of_week_labels, day_of_month_labels)]  
    # Concatenate labels

    ax.set_xticks(sorted_dates)
    ax.set_xticklabels(x_labels)
    ax.tick_params(axis='x', rotation=0)

plt.suptitle('Daily Mean Log Appliance Power Consumption')
fig.text(0, 0.5, 'Appliance Energy Consumption', va='center', rotation='vertical')
fig.text(0.5, 0, 'Day of the month', ha='center')
plt.tight_layout()
plt.savefig('Daily Mean Log Appliance Power Consumption.png')
plt.show();

Daily Consumption trends:
* Onset of the weekend: Upward trend in energy consumption
* Consistent downward trend in consumption levels at start of the week (pivot points highlighted in blue)

In [ ]:
df['hour'] = pd.to_datetime(df['date']).dt.strftime('%H').astype(int)

In [ ]:
WeekendDf = df[df.day_type == '1'].groupby('hour')['appliances'].median().reset_index()
WeekdayDf = df[df.day_type == '0'].groupby('hour')['appliances'].median().reset_index()


WeekendDf['appliances'] = np.exp(WeekendDf['appliances'])
WeekdayDf['appliances'] = np.exp(WeekdayDf['appliances'])

bar_width = 0.35

x = np.arange(len(WeekendDf['hour']))

fig, ax = plt.subplots(figsize=(15, 4))

ax.bar(x - bar_width/2, WeekendDf['appliances'], bar_width, label='Weekend')

ax.bar(x + bar_width/2, WeekdayDf['appliances'], bar_width, label='Weekday')

ax.set_xticks(x)
ax.set_xticklabels(WeekendDf['hour'])

ax.set_xlabel('Hour')
ax.set_ylabel('Appliances')
ax.set_title('Hourly Trends')

ax.legend()

plt.savefig(f'Weekend-vc-Weekday-hourly-trends.png')
plt.show()


Weekday vs Weekend Hourly Energy Consumption: Inferences
* Uptick in Energy consumption levels between 8AM to 9PM
* Peak Energy Consumption between the hours 5PM and 9PM
* Weekend daytime energy consumption is greater than that during weekdays.

**Ambient Morning Temperature (tout) tredline**

In [ ]:
def corr_map(df, title, method='spearman', numeric_only = True, tick_size = 10, annot_size=8, annot_kws=8):
    mask = np.triu(np.ones_like(df.corr(numeric_only=numeric_only), dtype=bool))
    heatmap = sns.heatmap(df.corr(method=method, numeric_only = numeric_only), fmt = '.2f',  annot = True, annot_kws = {'size': annot_kws}, mask=mask, cmap='BrBG')#
    plt.xticks(rotation=80, fontsize= tick_size)
    plt.yticks( fontsize= tick_size)

    heatmap.set_title(title, fontdict={'fontsize':annot_size}, pad=12)
    plt.show();

In [ ]:
df['inv_log_appliance'] = np.exp(df['appliances'])

In [ ]:
df['appliances'].max(), df['inv_log_appliance'].max()

In [ ]:
corr_map(df.drop(['month', 'date_2', 'time','rv1','rv2','visibility', 'windspeed',
       'week','day_num', 'day_type', 'week_num'], axis =1), method='pearson' , title='Feature Correlations')

<a id="3.3"></a>
## 3.3 Multivariate Analysis


In [ ]:
df['temprature_inside'] = df[['t1', 't2', 't3', 't4', 't5', 't7', 't8', 't9']].mean(axis=1)
df['temprature_outside'] = df['t6']

df['inv_log_appliances'] = np.exp(df['appliances']) - 30

MultvariateDF = df[['week_num', 'temprature_inside', 'temprature_outside', 'windspeed', 'inv_log_appliances']].groupby('week_num')[['temprature_inside', 'temprature_outside', 'windspeed', 'inv_log_appliances']].median().reset_index()
MultvariateDF['windspeed'] = MultvariateDF['windspeed'] * 6 +10

bar_width = 0.35
bar_offset = bar_width / 2

fig, ax = plt.subplots(figsize=(20,8)) 

plt.bar(MultvariateDF['week_num'], MultvariateDF['temprature_inside'], width=bar_width, align='center', label='temprature Inside')

plt.bar(np.array(MultvariateDF['week_num']) + bar_offset, MultvariateDF['temprature_outside'], width=bar_width, align='center', label='temprature Outside')


plt.plot(MultvariateDF['week_num'], MultvariateDF['windspeed'], label='Wind Speed', c = 'black', linestyle = 'dashed')

plt.plot(MultvariateDF['week_num'], MultvariateDF['inv_log_appliances'], label='Inv Log Appliances', linestyle = 'dotted')



plt.xlabel('Week Number')
plt.ylabel('')
plt.title('Appliances trends with temprature Inside and Outside')
plt.xticks(MultvariateDF['week_num'])

plt.legend()

plt.savefig(f'Appliances-trends-with-temprature-Inside-and-Outside.png')
plt.show()


In [ ]:
df['humidity_inside'] = df[['rh_1', 'rh_2', 'rh_3', 'rh_4', 'rh_5', 'rh_7', 'rh_8', 'rh_9']].mean(axis=1)
df['humidity_outside'] = df['rh_6']

df['inv_log_appliances'] = np.exp(df['appliances']) - 30

MultvariateDF = df[['week_num', 'humidity_inside', 'humidity_outside', 'windspeed', 'inv_log_appliances']].groupby('week_num')[['humidity_inside', 'humidity_outside', 'windspeed', 'inv_log_appliances']].median().reset_index()
MultvariateDF['windspeed'] = MultvariateDF['windspeed'] * 6 +10

bar_width = 0.35
bar_offset = bar_width / 2

fig, ax = plt.subplots(figsize=(20,8)) 

plt.bar(MultvariateDF['week_num'], MultvariateDF['humidity_inside'], width=bar_width, align='center', label='Humidity Inside')

plt.bar(np.array(MultvariateDF['week_num']) + bar_offset, MultvariateDF['humidity_outside'], width=bar_width, align='center', label='Humidity Outside')


plt.plot(MultvariateDF['week_num'], MultvariateDF['windspeed'], label='Wind Speed', c = 'black', linestyle = 'dashed')

plt.plot(MultvariateDF['week_num'], MultvariateDF['inv_log_appliances'], label='Log Inv Appliances', linestyle = 'dotted')



plt.xlabel('Week Number')
plt.ylabel('')
plt.title('Appliances trends with Humidity Inside and Outside')
plt.xticks(MultvariateDF['week_num'])

plt.legend()

plt.savefig(f'Appliances-trends-with-humidity-Inside-and-Outside.png')
plt.show()


* There is a **significant difference in mean inside temperature and outside temperature,** which suggests that the space is being **artificially heated.** This also follows our initial hypothesis from the weak correlation between internal temperature and internal humidity.
* We also observe that the **inverse log transformed target (appliance power usage at actuals) sharply follows the mean internal temperature changes.** This suggests that **internal  heating is a significant driver of household energy consumption.** We can save on consumption, if we use 
    - Energy efficient heating system
    - Better insulation 
    - Fixing air leakages from the interiors to the exteriors of the house

<a id="3.4"></a>
## 3.4 Summary Observations and Recommendations

**Observations**
Temperature columns - Temperature inside the house varies between 14.89 Deg & 29.85 Deg , temperatire outside (T6) varies between -6.06 Deg to 28.29 Deg . The reason for this variation is sensors are kept outside the house

**Humidiy columns** - Humidity inside house varies is between 20.60% to 63.36% with exception of RH_5 (Bathroom) and RH_6 (Outside house) which varies between 29.82% to 96.32% and 1% to 99.9% respectively.

**Appliances** - 75% of Appliance consumption is less than 100 Wh . With the maximum consumption of 1080 Wh (inverse log transformed target) , there will be outliers in this column and there are small number of cases where consumption is very high

**Lights column** - Intially we believed lights column will be able to give useful information . With 11438 0 (zero) enteries in 14801 rows , this column will not add any value to the model . We believed light consumption along with humidity level in a room will give idea about human presence in the room and hence its impact on Appliance consumption. Hence, we will dropping this column

<a id="4"></a>
# 4. Model Training, Hyperparameter tuning, Evaluation



In [ ]:
from sklearn import preprocessing, model_selection, metrics
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
from sklearn import neighbors
from sklearn.svm import SVR

import time
from math import sqrt
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
# 75% of the data is usedfor the training of the models and the rest is used for testing
train, test = train_test_split(df,test_size=0.25,random_state=40)

In [ ]:
# These features will be used to select the model, as it has train and test  split is available
feature_vars = col_temp + col_hum + col_weather +   col_time
target_vars = col_target


# We use below features after selecting the final model
model_features_vars = col_temp + col_hum + col_weather +  col_time
model_target_vars = col_target

In [ ]:
#Split training dataset into independent and dependent varibales
train_X = train[feature_vars]
train_y = train[target_vars]

#Split testing dataset into independent and dependent varibales
test_X = test[feature_vars]
test_y = test[target_vars]

# We use below features after selecting the final model
model_train_X = train[feature_vars]
model_train_y = train[target_vars]

In [ ]:
models = [
           ['Lasso', Lasso()],
           ['Ridge', Ridge()],
           ['K-Neighbors Regressor',  neighbors.KNeighborsRegressor()],
           ['SVR' , SVR(kernel='rbf')],
           ['Random Forest',RandomForestRegressor()],
           ['Extra Tree Regressor',ExtraTreesRegressor()],
           ['Gradient Boosting Classifier', GradientBoostingRegressor()] ,
           ['XGB Regressor', xgb.XGBRegressor()] ,
           ['MLP Regressor', MLPRegressor(  activation='relu', solver='adam',learning_rate='adaptive',max_iter=1000,learning_rate_init=0.01,alpha=0.01)]
         ]

In [ ]:
model_data = []
for name,curr_model in models :
    curr_model_data = {}
    curr_model.random_state = 78
    curr_model_data["Name"] = name
    start = time.time()
    curr_model.fit(train_X,train_y)
    end = time.time()
    curr_model_data["Train_Time"] = end - start
    curr_model_data["Train R2 Score"] = metrics.r2_score(train_y,curr_model.predict(train_X))
    curr_model_data["Validate R2 Score"] = metrics.r2_score(test_y,curr_model.predict(test_X))
    curr_model_data["Test RMSE Score"] = sqrt(mean_squared_error(test_y,curr_model.predict(test_X)))
    model_data.append(curr_model_data)

In [ ]:
modelComparison = pd.DataFrame(model_data)
modelComparison = modelComparison.sort_values(by=['Train R2 Score', 'Validate R2 Score', 'Test RMSE Score']).reset_index()
modelComparison

In [ ]:
modelComparison.set_index('Name', inplace=True)

In [ ]:
modelComparison[modelComparison['Train R2 Score'] > 0][['Train R2 Score', 'Validate R2 Score']].plot.bar()

# plt.figure(figsize = (8, 4))
plt.xlabel('')
plt.xticks(rotation = 45)
plt.ylabel('')
plt.title('Models Comparision')

plt.savefig(f'Models-comparision.png')
plt.show()

* By comparing all the model, found `Extra Tree Regressor` giving best results. 
* Further, we will implement hyperparameters tunning on `Extra Tree Regressor`
* Below are the best features, observed for `Extra Tree Regressor`

In [ ]:
xtr = ExtraTreesRegressor(criterion='poisson',max_depth=100,max_features='log2', n_estimators=350,n_jobs=-1,random_state=40)
xtr.fit(model_train_X,model_train_y)

In [ ]:
metrics.r2_score(test_y,xtr.predict(test_X))

Preparing the test Data for predictions

In [ ]:
df_t["hour"] = pd.to_datetime(df_t["date"]).dt.hour
df_t["weekday"] = pd.to_datetime(df_t["date"]).dt.weekday

In [ ]:
ids = list(df_t['ID'])
model_test_X = df_t[col_temp + col_hum + col_weather + col_time ]

In [ ]:
pred = xtr.predict(model_test_X)
sub_xtree = pd.DataFrame({"ID": ids, "appliances":list(pred)})
sub_xtree.to_csv("/kaggle/working/sub_xtree.csv", index=False)

In [ ]:
sub_xtree.head()

## **Achieved accuracy : 78.45 %**